In [2]:

import time
import itertools
import wget
import os
import png
import pickle
import numpy as np
import tmdbsimple as tmdb
import time
import sys
from PIL import Image , ImageFile

import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import precision_recall_curve
import scipy as sp

sns.set_style('white')
import pandas as pd


### Read Files

In [3]:
# Read list of balanced multi labels
LabList = pd.read_csv('genre_bal.csv')

# Read the file of movies filenames
PostList = pd.read_csv('PostersList_Dom.csv')

# Read Genres
GenresList = pd.read_csv('genresList.csv')

MovieId = LabList.ID.values

Len = len(MovieId)
print Len

4934


In [4]:
test = GenresList
test.head()

,Unnamed: 0,0
0,0,Action
1,1,Adventure
2,2,Animation
3,3,Comedy
4,4,Crime


In [5]:
LabNames = []

for i in range(len(test)):
    
    LabNames.append(test.iloc[i,1])
    
LabNames.append('id')
LabNames

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western',
 'id']

### Acquiring Features:


<br>

#### 1- Specifying labels


<br>

In [6]:
LabList[:10]

,Unnamed: 0,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,ID
0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,28932
1,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11563
2,3,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,23102
3,4,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,12628
4,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,16331
5,6,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,11658
6,7,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,13613
7,8,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,17664
8,9,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,25645
9,10,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,18806


#### 2- Transforming the Data

<br>

#### Function to  Handle Pixel Information

<br>

In [31]:
def image_vector(filename):
    
    # Open The image
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    post = Image.open(filename)

    # Transform it into a pixel Object
    pix = post.load()
    
    # Determine Size
    sz = post.size

    # Convert image to an array
    sample_ar = post.convert('RGB')

    # Construct a numpy array
    sample_array = np.array(sample_ar)
    
    # Construct vectors arrays for colors
    Red   = sample_array[:,:,0].ravel()
    Green = sample_array[:,:,1].ravel()
    Blue  = sample_array[:,:,2].ravel()
    
    # Reshape for aggregation
    Rresh = Red.reshape((1 , Red.shape[0]))
    Gresh = Green.reshape((1 , Green.shape[0]))
    Bresh = Blue.reshape((1 , Blue.shape[0]))
    
    # Concatenate
    Col1 = np.concatenate((Rresh , Gresh) , 0)
    Colvec = np.concatenate((Col1 , Bresh) , 0)
    
    
    return Colvec


In [32]:
PostList.head()

,Unnamed: 0,FileName,FilmId
0,0,./posters/22924.jpg,22924
1,1,./posters/14223.jpg,14223
2,3,./posters/27420.jpg,27420
3,5,./posters/12903.jpg,12903
4,6,./posters/23398.jpg,23398


In [33]:
# Extracting the ids vector
mvid = LabList['ID'].values

# Initiate the features array
features = []

# Extract the features
for i in range (1):
    
    # Extract the poster file
    posterfile = PostList['FileName'].iloc[i]
    
    # Extract the movie id
    mvid = PostList['FilmId'].iloc[i]
    
    # Calculate the poster vector
    poster = image_vector(posterfile)
    
    # Initiate movie feature list
    sing = []
    
    # Add Mean of RGB
    sing.append(poster.mean(1)[0])
    sing.append(poster.mean(1)[1])
    sing.append(poster.mean(1)[2])
    
    # Add Standard deviation
    sing.append(poster.std(1)[0])
    sing.append(poster.std(1)[1])
    sing.append(poster.std(1)[2])
    
    # Calculate the correlation distance
    dist = sp.spatial.distance.pdist(poster , 'correlation')
    
    # Append the distance component
    sing.append(dist[0])
    sing.append(dist[1])
    sing.append(dist[2])
    
    # Add movie ID
    sing.append(mvid)
    
    # Append to the features array
    features.append(sing)
    
    time.sleep(0.1)
    
    sys.stdout.flush()
    sys.stdout.write("\r Step: {} , id {}".format(i , mvid))


(763L, 500L, 3L)
 Step: 0 , id 22924




<br>
#### Construct feature array

<br>

In [13]:
# Construct the array
featarray = np.array(features)

featarray.shape

(10L, 10L)



<br>

### Extract to Files

<br>

In [ ]:
# 
X_raw = featarray
xdf = pd.DataFrame(featarray , columns = ['mR' , 'mG' , 'mB' , 'stdR' , 'stdG' , 'stdB' , 'D1' , 'D2' , 'D3' . 'ID'])
xdf.to_csv('Features_Balanced.csv')
# DataAll = pd.concat([xdf , PostList.FilmId] , 1)
# DataAll.to_csv('Databalanced.csv')